# NLU Group 1: Final Project
## Example: Fine-tuning a classifier on the Social Bias Inference Corpus

Based on: https://github.com/huggingface/notebooks/blob/master/examples/text_classification.ipynb

This notebook by: Cameron Clarke (ccc779@nyu.edu)

Created: 3/16/2021

# 0. Setup

In [ ]:
!pip install transformers
!pip install datasets
!pip install bert_score
!pip install optuna

# 1. Preprocessing

In [ ]:
from transformers import AutoTokenizer
import datasets
from datasets import load_dataset, load_metric
import numpy as np
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
# [TODO] update this for whichever model type is desired

model_checkpoint = "bert-base-cased"
batch_size = 8

In [ ]:
# [TODO] update this for whichever dataset is desired

dataset = load_dataset("social_bias_frames")

Using custom data configuration default


Dataset social_bias_frames downloaded and prepared to /root/.cache/huggingface/datasets/social_bias_frames/default/0.0.0/7ccf5e07dabdba6791693ea27289996d4771f586aa88f1ff05c52645f2cfd41d. Subsequent calls will reuse this data.


In [ ]:
metric = load_metric('bertscore')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
sentence1_key = 'post'

In [ ]:
def preprocess_function(examples, sentence1_key, sentence2_key=None):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True, padding=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True, padding=True)

In [ ]:
label_colname = 'offensiveYN'

relabel_dict = {
    '0.0': 0, # not offensive
    '0.5': 1, # maybe offiensive
    '1.0': 2, # offensive
    '': None # missing value
}

relabel_func = lambda column: [relabel_dict[elt] for elt in column]

In [ ]:
# Following a design pattern found here: https://huggingface.co/transformers/custom_datasets.html#using-the-nlp-datasets-metrics-library


encoded_dataset = dataset.map(lambda x: preprocess_function(x, sentence1_key),  batched=True)
encoded_dataset = encoded_dataset.map(lambda x: {'labels': relabel_func(x[label_colname])},  batched=True)

new_features = encoded_dataset['train'].features.copy()
new_features["labels"] = datasets.ClassLabel(names=['no', 'maybe', 'yes'])

encoded_dataset['train'] = encoded_dataset['train'].cast(new_features)
encoded_dataset['validation'] = encoded_dataset['validation'].cast(new_features)
encoded_dataset['test'] = encoded_dataset['test'].cast(new_features)



In [ ]:
encoded_dataset = encoded_dataset.filter(lambda row: not (row['labels'] is None))

In [ ]:
encoded_dataset['train']['offensiveYN'], encoded_dataset['train']['labels']

(['1.0',
  '0.5',
  '0.5',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '0.0',
  '0.0',
  '0.0',
  '1.0',
  '1.0',
  '0.5',
  '0.5',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '0.0',
  '1.0',
  '1.0',
  '0.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '0.5',
  '0.5',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '0.0',
  '1.0',
  '0.0',
  '1.0',
  '1.0',
  '0.0',
  '0.5',
  '0.0',
  '1.0',
  '1.0',
  '0.0',
  '0.5',
  '0.5',
  '1.0',
  '0.0',
  '1.0',
  '0.5',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '0.5',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '0.0',
  '0.0',
  '1.0',
  '0.0',
  '1.0',
  '0.5',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
  '0.5',
  '0.0',
  '0.0',
  '0.0',
  '0.0',
  '0.5',
  '1.0',
  '1.0',
  '1.0',
  '1.0',
 

# 2. Fine-tuning

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer


num_labels = 3 # The original 'offensiveYN' outcome is a categorical variable with three levels
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
model.eval()
model.to('cuda')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# [TODO] update this with whatever metric is desired

metric_name = "accuracy"

args = TrainingArguments(
    "test-SBIC-bert",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [ ]:
# Reference: https://github.com/armandalewis/ling-ga-1012/blob/main/eval_metrics.py

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        # 'f1': f1,
        # 'precision': precision,
        # 'recall': recall
    }

# def compute_metrics(eval_pred, metric):
#     predictions, labels = eval_pred

#     predictions = np.argmax(predictions, axis=1)
#     return metric.compute(predictions=predictions, references=labels)

In [ ]:
# validation_key = "validation"
# trainer = Trainer(
#     model,
#     args,
#     train_dataset=encoded_dataset['train'],
#     eval_dataset=encoded_dataset['validation'],
#     tokenizer=tokenizer,
#     compute_metrics=lambda x: compute_metrics(x, metric)
# )

# Hyperparameter Tuning

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
    # compute_metrics=lambda x: compute_metrics(x, metric)
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

[I 2021-03-17 23:09:13,041] A new study created in memory with name: no-name-3e809fb6-99ae-457d-8b14-70d3f1643ce4
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassi

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,0.694900,0.620004,0.794266,54.848500,300.774000
2,0.656000,0.661907,0.797781,55.120800,299.288000


[I 2021-03-18 00:30:46,401] Trial 0 finished with value: 0.7977814148026914 and parameters: {'learning_rate': 1.9947717077303566e-06, 'num_train_epochs': 2, 'seed': 23, 'per_device_train_batch_size': 4}. Best is trial 0 with value: 0.7977814148026914.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the chec

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,0.628000,0.601337,0.776080,55.641900,296.485000
2,0.581600,0.583922,0.783173,55.267600,298.493000
3,0.535300,0.581967,0.783961,55.351500,298.041000


[I 2021-03-18 02:22:17,569] Trial 1 finished with value: 0.7839607201309329 and parameters: {'learning_rate': 1.413983402378036e-06, 'num_train_epochs': 3, 'seed': 9, 'per_device_train_batch_size': 8}. Best is trial 0 with value: 0.7977814148026914.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkp

RuntimeError: ignored

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

In [ ]:
# trainer.evaluate()